### Prepare maps for analysis

#### Hypotheses:

Parametric effect of gain:

1. Positive effect in ventromedial PFC - for the equal indifference group
2. Positive effect in ventromedial PFC - for the equal range group
3. Positive effect in ventral striatum - for the equal indifference group
4. Positive effect in ventral striatum - for the equal range group

Parametric effect of loss:
- 5: Negative effect in VMPFC - for the equal indifference group
- 6: Negative effect in VMPFC - for the equal range group
- 7: Positive effect in amygdala - for the equal indifference group
- 8: Positive effect in amygdala - for the equal range group

Equal range vs. equal indifference:

- 9: Greater positive response to losses in amygdala for equal range condition vs. equal indifference condition.

#### Notes on data

Data and hypothesis decisions were obtained from 70 teams.  Of these, the maps for 17 teams were excluded from further analysis due to the following problems:

##### Bad normalization/resampling:

- SVXXBBHN_98BT: rejected due to very bad normalization
- UGXECSGG_L1A8: resampled image much smaller than template brain
- VDOVGCPL_4SZ2: resampled image offset from template brain
- WBTVHMSS_4TQ6: resampled image  offset and too large compared to template


##### Missing thresholded images:

- WIYTWEEA_2T7P: missing thresholded images


##### Used surface-based analysis (only provided data for cortical ribbon:

- DRVQPPNO_1K0E
- UMSEIVRB_X1Z4 


##### Bad unthresholded images:

- TMZGNUWI_R5K7: "unthresholded" images appeared to be thresholded



##### Missing data:

- YWGVKXBZ_6FH5: missing much of the central brain
- ACMKXFTG_P5F3: rejected due to large amounts of missing data across brain
- NNUNPWLT_0H5E: rejected due to large amount of missing brain in center
- CMJIFMMR_L3V8: rejected due to large amount of missing brain in center


##### Bad histograms:  
visual examiniation of histograms of unthresholded images showed a number of them that were clearly not distribted as z/t

- RIIVGRDK_E3B6: very long tail, with substantial inflation at a value just below zero
- JAWCZRDS_V55J: very small values
- EPBGXICO_I07H: bimodal, with second distribution centered around 2.5
- UXJVRRRR_0C7Q: appears to be a p-value distribution, with slight excursions below and above zero
- OXLAIRNK_Q58J: bimodal, zero-inflated with a second distribution centered around 5




In [ ]:
# all of the functionality is now embedded in the Narps() class within narps.py
import os,sys,glob,warnings
import matplotlib.pyplot as plt
import numpy
import nilearn.input_data

from narps import Narps
from utils import get_masked_data

overwrite = False

In [ ]:
# set up directories
basedir='/Users/poldrack/data_unsynced/NARPS'

# instantiate main Narps class, which loads data
# to rerun everything, set overwrite to True

narps = Narps(basedir,overwrite=overwrite)

output_dir = narps.dirs.dirs['output']
mask_img = narps.dirs.MNI_mask

In [ ]:
# load cached data

# narps.load_data()

### Create binarized thresholded masks

load the thresholded maps and binarize the absolute value (to detect either positive or negative signals).  save output to the *thresh_mask_orig* directory (in original space).

In [ ]:
narps.get_binarized_thresh_masks()


### Resample images

The images were supposed to be submitted in MNI space.  Here we use nilearn.image.resample_to_img() to resample the image into the FSL MNI space (91x109x91, 2mm isotropic).  For unthresholded maps, we use continuous interpolation; for thresholded maps, we use linear interpolation and then threshold at 0.5; this avoids some errors that occurred using nearest neighbor interpolation, where there were empty voxels in some slices. save to *resampled* directory. 

In [ ]:
narps.get_resampled_images()


### Check image values

Compute number of NA and zero voxels for each image, and save to image_metadata_df.csv.

In [ ]:
image_value_df = narps.check_image_values()


### Combine resampled images 

Combine all images into a single concatenated file for each hypothesis.  save to *{thresh,unthresh}\_concat\_resampled*.

In [ ]:
narps.create_concat_images(datatype='resampled')

### Rectify maps

Rectify the unthresholded maps if they used positive values for negative activations - so that all unthresholded maps reflect positive values for the hypothesis in question.  We check this automatically by comparing the thresholded and unthresholded maps - if all voxels within the thresholded map are negative, then we multiply the unthresholded map by -1.  Save to *rectified*


In [ ]:
narps.create_rectified_images()

### Create concatenated rectified unthresholded images

In [ ]:
narps.create_concat_images(datatype='rectified',imgtypes = ['unthresh'])

### Compute image statistics

Compute range and standard deviation maps across teams for unthresholded/rectified maps. 

In [ ]:
narps.compute_image_stats()

### Convert all unthresholded images to Z stat images

Use metadata reported by teams (contained in NARPS_analysis_teams_members.csv) which specifies what kind of unthresholded statistical images were submitted.  For those who reported T values, convert those to Z values using Hughett's transform (based on https://github.com/vsoch/TtoZ/blob/master/TtoZ/scripts.py)

In [ ]:
narps.convert_to_zscores()

### Estimate smoothness

Use FSL's SmoothEstimate (via nipype) to estimate smoothness of each Z map.

In [ ]:
smoothness_df = narps.estimate_smoothness()

### Save outputs

Serialize the important information from the class and save to *narps_prepare_maps.pkl*


In [ ]:
output = narps.write_data()

## Make reports

Create some diagnostic reports

### Make report showing all orig maps with threshold overlays

This report includes all maps for which data were available, including those that were excluded


In [ ]:
cut_coords = [-24,-10,4,18,32,52,64]

bins = numpy.linspace(-5,5)


figdir = os.path.join(output_dir,'figures/orig_map_overlays')

if not os.path.exists(figdir):
    os.mkdir(figdir)


for hyp in [1,2,5,6,7,8,9]:
    outfile = os.path.join(figdir,'hyp%d_orig_map_overlays.pdf'%hyp)
    
    if not os.path.exists(outfile) or overwrite:
        print('making map overlay figure for hyp',hyp)
        
        # find all maps
        hmaps = glob.glob(os.path.join(output_dir,'orig/*_*'))
        
        collection_ids = [os.path.basename(i) for i in hmaps]
        collection_ids.sort()

        fig, ax = plt.subplots(len(collection_ids),2,figsize=(len(collection_ids),140),gridspec_kw={'width_ratios': [2, 1]})
        ctr=0

        for collection_id in collection_ids:
            teamID = collection_id.split('_')[1]
            unthresh_img=os.path.join(output_dir,'orig/%s/hypo%d_unthresh.nii.gz'%(collection_id,hyp))
            thresh_img=os.path.join(output_dir,'thresh_mask_orig/%s/hypo%d_thresh.nii.gz'%(collection_id,hyp))
            
            if not (os.path.exists(thresh_img) or os.path.exists(unthresh_img)):
                print('skipping',teamID)
                continue
                
            if not teamID in narps.complete_image_sets:
                imagetitle = '%s (excluded)'%teamID
            else:
                imagetitle = teamID
                
            display = nilearn.plotting.plot_stat_map(unthresh_img, display_mode="z", 
                    colorbar=True,title=collection_id,
                                      cut_coords = cut_coords,axes = ax[ctr,0],cmap='gray')

            with warnings.catch_warnings():  # ignore levels warning
                warnings.simplefilter("ignore")
                display.add_contours(thresh_img, filled=False, alpha=0.7, levels=[0.5], colors='b') 

            masker=nilearn.input_data.NiftiMasker(mask_img=thresh_img)
            maskdata = masker.fit_transform(unthresh_img)
            if numpy.sum(maskdata)>0:  # check for empty mask
                hist_result=ax[ctr,1].hist(maskdata,bins=bins)
            ctr+=1
        plt.savefig(outfile)
        plt.close(fig)


#### Create histograms for in-mask values in unthresholded images

These are only created for the images that were successfully registered and rectified.

In [ ]:
figdir = os.path.join(output_dir,'figures/unthresh_histograms')

if not os.path.exists(figdir):
    os.mkdir(figdir)

for hyp in [1,2,5,6,7,8,9]:
    outfile = os.path.join(figdir,'hyp%d_unthresh_histogram.pdf'%hyp)
    
    if not os.path.exists(outfile) or overwrite:
        print('making figure for hyp',hyp)
        unthresh_data,labels=get_masked_data(hyp,mask_img,output_dir,imgtype='unthresh',dataset='rectified')
        fig, ax = plt.subplots(int(numpy.ceil(len(labels)/3)),3,figsize=(16,50))
        
        # make three columns - these are row and column counters
        ctr_x=0
        ctr_y=0
        
        for i,l in enumerate(labels):
                ax[ctr_x,ctr_y].hist(unthresh_data[i,:],100)
                ax[ctr_x,ctr_y].set_title(l)
                ctr_y+=1
                if ctr_y>2:
                    ctr_y=0
                    ctr_x+=1
        plt.tight_layout()

        plt.savefig(outfile)
